## 테스트 데이터 로드 및 전처리

In [3]:
import torch
from datasets import load_from_disk
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [4]:
test_dataset = load_from_disk('test_dataset')

FileNotFoundError: Directory test_dataset not found

In [ ]:
# base_model tokenizer load
model = 'meta-llama/Llama-3.1-8B'
tokenizer = AutoTokenizer.from_pretrained(model)

테스트 데이터를 불러와서 입력과 레이블 형태로 전처리합니다.

In [ ]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

임의로 20번 샘플을 출력합니다.

In [ ]:
# 20번 테스트 샘플 출력
print(prompt_lst[20])

In [ ]:
# 임의의 20번 테스트 샘플 레이블 출력
print(label_lst[20])

## 기본 모델

In [ ]:
# pipeline 객체와 LLM의 입력을 넣으면 LLM의 답변을 출력해주는 함수
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [ ]:
model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2-7B-Instruct', device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
eos_token = tokenizer("<|im_end|>", add_special_tokens=False)["input_ids"][0]

기본 모델에 임의로 20번 샘플을 넣어서 출력합니다. 기본 모델은 출처를 남기지 않습니다.

In [ ]:
result = test_inference(pipe, prompt_lst[20])
print(result)

기본 모델에 임의로 400~404번 샘플을 출력합니다. 기본 모델은 지시사항에서 출처를 남기라는 지시를 따르지 않습니다.

In [ ]:
for prompt, label in zip(prompt_lst[400:405], label_lst[400:405]):
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

## 튜닝 모델

In [ ]:
peft_model_id = "qwen2-7b-rag-ko/checkpoint-177"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
fine_tuned_pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)
eos_token = tokenizer("<|im_end|>", add_special_tokens=False)["input_ids"][0]

튜닝 모델에 임의로 20번 샘플을 넣어서 출력합니다. 튜닝 모델은 출처를 남깁니다.

In [ ]:
result = test_inference(fine_tuned_pipe, prompt_lst[20])
print(result)

튜닝 모델에 임의로 400~404번 샘플을 출력합니다. 튜닝 모델은 튜닝이 되었으므로 프롬프트 지시사항에 따라서 출처를 남기는 모습을 보여줍니다.  
물론, 출처를 남기지 않는 샘플들도 존재할 것입니다. 이는 학습이 덜 된 것이므로 학습 데이터의 비율을 높이고 에포크를 높이시기 바랍니다.

In [ ]:
for prompt, label in zip(prompt_lst[400:405], label_lst[400:405]):
    print(f"    response:\n{test_inference(fine_tuned_pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

## Merge

In [ ]:
!python merge.py \
    --base_model_name_or_path Qwen/Qwen2-7B-Instruct \
    --peft_model_path ./qwen2-7b-rag-ko/checkpoint-177 \
    --output_dir ./output_dir

In [ ]:
# 실습 후에는 상단의 Kernel > Shut Down Kernel을 눌러서 종료합니다.